<a href="https://colab.research.google.com/github/TonyDql/summer2023_educ_proj/blob/main/data_cleaning_qilong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Read the csv file
df = pd.read_csv('/content/drive/MyDrive/rawdata_longform_full_08-19-22.csv')


In [5]:
# Column names
print(df.columns)

Index(['ID', 'Global_timestamp', 'Timestamp', 'Event type', 'Event', 'Level',
       'Details', 'filename', 'epoch_time', 'object_orientation',
       'x_coordinate', 'y_coordinate', 'objects'],
      dtype='object')


In [6]:
# First 20 rows as example
print(df.head(20))

     ID Global_timestamp  Timestamp Event type          Event  \
0   100        0:0:0:000  0:0:0:000      event  session_start   
1   100        0:0:9:583  0:0:0:000      event          start   
2   100        0:0:9:583  0:0:0:000      event    init_object   
3   100        0:0:9:583  0:0:0:000      event    init_object   
4   100        0:0:9:583  0:0:0:000      event    init_object   
5   100        0:0:9:583  0:0:0:000      event    init_object   
6   100        0:0:9:583  0:0:0:000      event    init_object   
7   100        0:0:9:583  0:0:0:000      event    init_object   
8   100        0:0:9:583  0:0:0:000      event    init_object   
9   100        0:0:9:583  0:0:0:000      event    init_object   
10  100        0:0:9:583  0:0:0:000      event    init_object   
11  100        0:0:9:583  0:0:0:000      event    init_object   
12  100        0:0:9:583  0:0:0:000      event    init_object   
13  100        0:0:9:583  0:0:0:000      event    init_object   
14  100        0:0:9:583 

Below is to drop all the rows with Event value ('session_start', 'start', 'init_object')

In [7]:
# Filter dataframe by dropping all the rows with too few information on the events
events_to_drop = ['session_start', 'start', 'init_object']
# df1 is the first filtered data
df1 = df[~df['Event'].isin(events_to_drop)]
print(df1.head(10))
df1 = df1.sort_values('ID')

     ID Global_timestamp   Timestamp Event type     Event               Level  \
69  100        0:0:9:583   0:0:0:000      event  rule_add  0level:baba is you   
70  100        0:0:9:583   0:0:0:000      event  rule_add  0level:baba is you   
71  100        0:0:9:583   0:0:0:000      event  rule_add  0level:baba is you   
72  100        0:0:9:583   0:0:0:000      event  rule_add  0level:baba is you   
73  100       0:0:55:300  0:0:45:716     change    update  0level:baba is you   
74  100       0:0:55:300  0:0:45:716      input     right  0level:baba is you   
75  100       0:0:55:466  0:0:45:883     change    update  0level:baba is you   
76  100       0:0:55:466  0:0:45:883      input     right  0level:baba is you   
77  100       0:0:55:633  0:0:46:050     change    update  0level:baba is you   
78  100       0:0:55:633  0:0:46:050      input     right  0level:baba is you   

                Details   filename     epoch_time  object_orientation  \
69    12:7:baba is you   100_0.txt 

In [122]:
# df2 is to take only the first n rows of df1 to make a demo so that the computation for testing can be quicker
df2 = df1.head(200000)
# print(df2)


In [123]:
# task_id is a new column that indicate the uniqueness of each ID with each level
# Meaning, either the change of ID or level will cause a change in task_id
track_id = 1
track_task_id = 1
level_list = list()
task_id_list = list()
max_task_id = 0
for index, row in df2.iterrows():
  if row['Level'] not in level_list and row['ID'] == track_id:
    task_id_list.append(track_task_id + max_task_id)
    track_task_id = track_task_id + 1
    level_list.append(row['Level'])
  elif row['Level'] in level_list and row['ID'] == track_id:
    task_id_list.append(level_list.index(row['Level']) + 1 + max_task_id)
  if row['ID'] != track_id:
    level_list = list()
    track_id = row['ID']
    max_task_id = max(task_id_list)
    task_id_list.append(track_task_id + max_task_id)
    track_task_id = track_task_id + 1
    level_list.append(row['Level'])

In [124]:
# df3 is the dataframe with the new column "task_id"
df3 = df2.copy()
df3['task_id'] = task_id_list
# print(df3.tail(50)[['ID', 'Level', 'task_id']])

In [125]:
df3.head(10)

,ID,Global_timestamp,Timestamp,Event type,Event,Level,Details,filename,epoch_time,object_orientation,x_coordinate,y_coordinate,objects,task_id
5791297,1,0:15:8:533,0:4:11:983,change,update,6level:grass yard,flag:17:3:0,1_2.txt,1636286131533,0.0,17,3,flag,1
5781870,1,0:14:59:900,0:2:15:133,change,update,3level:out of reach,baba:9:12:3,1_1.txt,1636283930900,3.0,9,12,baba,2
5781869,1,0:14:59:900,0:2:15:133,change,update,3level:out of reach,rock:9:13:3,1_1.txt,1636283930900,3.0,9,13,rock,2
5781868,1,0:14:59:900,0:2:15:133,change,update,3level:out of reach,baba:9:11:3,1_1.txt,1636283930900,3.0,9,11,baba,2
5781867,1,0:14:59:433,0:2:14:666,input,left,3level:out of reach,NaN,1_1.txt,1636283930433,NaN,NaN,NaN,NaN,2
5781866,1,0:14:59:433,0:2:14:666,change,update,3level:out of reach,baba:9:11:2,1_1.txt,1636283930433,2.0,9,11,baba,2
5781865,1,0:14:59:433,0:2:14:666,change,update,3level:out of reach,baba:10:11:2,1_1.txt,1636283930433,2.0,10,11,baba,2
5781864,1,0:14:59:083,0:2:14:316,input,down,3level:out of reach,NaN,1_1.txt,1636283930083,NaN,NaN,NaN,NaN,2
5781863,1,0:14:59:083,0:2:14:316,change,update,3level:out of reach,baba:10:11:3,1_1.txt,1636283930083,3.0,10,11,baba,2
5781862,1,0:14:58:550,0:2:13:783,input,right,3level:out of reach,NaN,1_1.txt,1636283929550,NaN,NaN,NaN,NaN,2


In [137]:
from collections import Counter

# filtered_df is the dataframe without all rows that has task_id counts that is less that n (currently n=2)
# Meaning we droped the rows that has insufficient data that can make a conclusion about that ID at one level

filter_list = list()
count_dict = Counter(task_id_list)
for num, count in count_dict.items():
    # print(f"Number {num} appears {count} times")
    if count < 40:
      filter_list.append(num)
filtered_df = df3[~df3['task_id'].isin(filter_list)]



In [138]:
# first sort by task_id to make it a more organized visulization
# Then create a column "win_status" that is win if even = win and nan otherwise

filtered_df = filtered_df.sort_values('task_id')
filtered_df['win_status'] = np.where(filtered_df['Event'] == "win", "win", np.nan)


In [139]:
# fills missing values in the 'win_status' column with the nearest non-null value within each group of 'task_id'.

filtered_df['win_status'] = filtered_df.groupby('task_id')['win_status'].transform(lambda x: x.ffill().bfill())

In [160]:
# For each group of task_id, if there are more than one win, remove all rows after the first win
grouped = filtered_df.groupby('task_id')
time_format = "%H:%M:%S:%f"
group_list = list()

# temp_finished_df is the new dataframe that store the original dataframe with above modification
temp_finished_df = pd.DataFrame()
for _, group in grouped:
  # print("before: ", group.shape[0])
  group_list.append(group.shape[0])
  win_rows = group[group['Event'] == "win"]
  if win_rows.shape[0] > 1:
    min_win = win_rows.iloc[0]
    for index, row in win_rows.iterrows():
      time1 = pd.to_datetime(row['Timestamp'], format=time_format)
      time2 = pd.to_datetime(min_win['Timestamp'], format=time_format)
      if time1 < time2:
        min_win = row
    time_group = pd.to_datetime(group['Timestamp'], format=time_format)
    time_min_win = pd.to_datetime(min_win['Timestamp'], format=time_format)
    group = group[time_group <= time_min_win]
    if group[group['win_status'] == "win"].shape[0] > 1:
      group = group.drop_duplicates(subset='Timestamp', keep='first')
  temp_finished_df = pd.concat([temp_finished_df ,group])
  # print("after: " , group.shape[0])


199659
180293
